## Analysis Summary

My friends and I are planning a vacation after Covid. We have 3 states(MA/FL/TX) in mind. 
   - Based on Yelp datasets, when we visit MA/FL/TX, we should definitely try Neptune Oyster/Café Tu Tu Tango/Franklin Barbecue respectively
       - 5 latest positive reviews and tips comments
   - Based on Yelp datasets, when we visit MA/FL/TX, if we want to have coffee, we should definitely check out Mike & Patty's/Se7en Bites/Bouldin Creek Cafe respectively
       - 5 latest positive reviews and tips comments

### Connect to an Azure Blob Storage Account

In [0]:
spark.conf.set(
  "fs.azure.account.key.ivyprojectmovie.blob.core.windows.net",
  "+z/sopCfJqbq0DURX9seAUXcbf7LB1/1SqyWohUF2Wyi9eQXu5csVrEhCnyRx7McF2p9oBxob2L/Y7m68iVLeA==")
dbutils.fs.ls("wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/")

Out[39]: [FileInfo(path='wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_business.json', name='yelp_academic_dataset_business.json', size=124380583),
 FileInfo(path='wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_business.parquet/', name='yelp_academic_dataset_business.parquet/', size=0),
 FileInfo(path='wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_checkin.json', name='yelp_academic_dataset_checkin.json', size=398272056),
 FileInfo(path='wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_checkin.parquet/', name='yelp_academic_dataset_checkin.parquet/', size=0),
 FileInfo(path='wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_review.json', name='yelp_academic_dataset_review.json', size=6936678061),
 FileInfo(path='wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_review.parquet/', name='yelp_academic_dataset_review.parquet/', size=0),
 FileInfo(path='wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_tip.json', name='yelp_academic_dataset_tip.json', size=230307244),
 FileInfo(path='wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_tip.parquet/', name='yelp_academic_dataset_tip.parquet/', size=0),
 FileInfo(path='wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_user.json', name='yelp_academic_dataset_user.json', size=3684505303),
 FileInfo(path='wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_user.parquet/', name='yelp_academic_dataset_user.parquet/', size=0)]

### Import Libraries

In [0]:
from pyspark.sql.functions import rank, col
import pyspark.sql.functions as f
from pyspark.sql.window import Window

### Read Datasets and Convert to Parquet for Better Performance

In [0]:
# business
# df_business = spark.read.json("wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_business.json")
# df_business.write.mode('overwrite').parquet("wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_business.parquet")
df_business = spark.read.parquet("wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_business.parquet")

# tip
# df_tip = spark.read.json("wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_tip.json")
# df_tip.write.mode('overwrite').parquet("wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_tip.parquet")
df_tip = spark.read.parquet("wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_tip.parquet")

# review
# df_review = spark.read.json("wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_review.json")
# df_review.write.mode('overwrite').parquet("wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_review.parquet")
df_review = spark.read.parquet("wasbs://yelp-dataset@ivyprojectmovie.blob.core.windows.net/yelp_academic_dataset_review.parquet")

### Clean & Reformat Datasets

In [0]:
nested_columns = []
for c in df_business.dtypes:
  name, col_type = c[0], c[1][:6]
  if col_type == 'struct':
    nested_columns.append(c[0])
print(nested_columns)  

exploded_columns = []
for nested_col in nested_columns:
  exploded_df = df_business.select(nested_col + ".*")
  exploded_columns.append(exploded_df)
exploded_columns

['attributes', 'hours']
Out[42]: [DataFrame[AcceptsInsurance: string, AgesAllowed: string, Alcohol: string, Ambience: string, BYOB: string, BYOBCorkage: string, BestNights: string, BikeParking: string, BusinessAcceptsBitcoin: string, BusinessAcceptsCreditCards: string, BusinessParking: string, ByAppointmentOnly: string, Caters: string, CoatCheck: string, Corkage: string, DietaryRestrictions: string, DogsAllowed: string, DriveThru: string, GoodForDancing: string, GoodForKids: string, GoodForMeal: string, HairSpecializesIn: string, HappyHour: string, HasTV: string, Music: string, NoiseLevel: string, Open24Hours: string, OutdoorSeating: string, RestaurantsAttire: string, RestaurantsCounterService: string, RestaurantsDelivery: string, RestaurantsGoodForGroups: string, RestaurantsPriceRange2: string, RestaurantsReservations: string, RestaurantsTableService: string, RestaurantsTakeOut: string, Smoking: string, WheelchairAccessible: string, WiFi: string],
 DataFrame[Friday: string, Monday: string, Saturday: string, Sunday: string, Thursday: string, Tuesday: string, Wednesday: string]]

In [0]:
# source is from https://docs.microsoft.com/en-us/azure/synapse-analytics/how-to-analyze-complex-schema

from pyspark.sql.functions import col

def flatten_df(nested_df):
    stack = [((), nested_df)]
    columns = []

    while len(stack) > 0:
        parents, df = stack.pop()
        
        #exploded, non-struct datatype will be kept
        flat_cols = [
            col(".".join(parents + (c[0],))).alias("_".join(parents + (c[0],)))
            for c in df.dtypes
            if c[1][:6] != "struct"
        ]
        columns.extend(flat_cols)
        
        nested_cols = [
            c[0]
            for c in df.dtypes
            if c[1][:6] == "struct"
        ]
      
        for nested_col in nested_cols:
            projected_df = df.select(nested_col + ".*")
            stack.append((parents + (nested_col,), projected_df))

    return nested_df.select(columns)


df_exploded_business = flatten_df(df_business)
display(df_exploded_business.limit(10))

address,business_id,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars,state,hours_Friday,hours_Monday,hours_Saturday,hours_Sunday,hours_Thursday,hours_Tuesday,hours_Wednesday,attributes_AcceptsInsurance,attributes_AgesAllowed,attributes_Alcohol,attributes_Ambience,attributes_BYOB,attributes_BYOBCorkage,attributes_BestNights,attributes_BikeParking,attributes_BusinessAcceptsBitcoin,attributes_BusinessAcceptsCreditCards,attributes_BusinessParking,attributes_ByAppointmentOnly,attributes_Caters,attributes_CoatCheck,attributes_Corkage,attributes_DietaryRestrictions,attributes_DogsAllowed,attributes_DriveThru,attributes_GoodForDancing,attributes_GoodForKids,attributes_GoodForMeal,attributes_HairSpecializesIn,attributes_HappyHour,attributes_HasTV,attributes_Music,attributes_NoiseLevel,attributes_Open24Hours,attributes_OutdoorSeating,attributes_RestaurantsAttire,attributes_RestaurantsCounterService,attributes_RestaurantsDelivery,attributes_RestaurantsGoodForGroups,attributes_RestaurantsPriceRange2,attributes_RestaurantsReservations,attributes_RestaurantsTableService,attributes_RestaurantsTakeOut,attributes_Smoking,attributes_WheelchairAccessible,attributes_WiFi
13 Bessom St,wlCwgeZgAr3Nz5etwivB2A,"Pets, Pet Groomers, Pet Services",Marblehead,1,42.5006520345,-70.8602147163,All Paws on Deck,01945,5,5.0,MA,9:0-17:0,0:0-0:0,9:0-17:0,null,9:0-17:0,9:0-17:0,null,null,null,null,null,null,null,null,null,null,null,null,False,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
300 Airport Rd,pme8Sq7O_cNnZv4kP9EzWw,"Food, Coffee & Tea, Donuts",East Boston,1,42.3660592,-71.0163073,Dunkin',02128,8,3.0,MA,5:0-22:0,5:0-22:0,5:0-22:0,5:0-22:0,5:0-22:0,5:0-22:0,5:0-22:0,null,null,null,null,null,null,null,False,null,True,"{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}",null,False,null,null,null,null,None,null,null,null,null,null,null,null,null,null,None,null,null,True,null,1,null,null,True,null,null,'free'
54 Church St,NVmRRoIfMet-vH4Qt7UhdA,"Fashion, Shopping, Accessories, Jewelry",Cambridge,0,42.3740244,-71.1208505,Tis Tik,02138,25,4.5,MA,10:30-21:0,10:30-20:0,10:30-21:0,11:0-18:0,10:30-21:0,10:30-20:0,10:30-20:0,null,null,null,null,null,null,null,True,null,True,"{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",False,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2,null,null,null,null,null,null
2040 Stringtown Rd,qfuIvXCy5ew_5IYp3nT4_w,"American (Traditional), Burgers, Restaurants, American (New), Breakfast & Brunch",Grove City,1,39.8789611,-83.057491,IHOP,43123,57,2.0,OH,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,null,null,u'none',"{'touristy': None, 'hipster': None, 'romantic': False, 'divey': None, 'intimate': None, 'trendy': None, 'upscale': None, 'classy': None, 'casual': True}",null,null,null,False,null,True,"{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",False,null,False,null,null,null,null,null,True,"{'dessert': None, 'latenight': None, 'lunch': True, 'dinner': None, 'brunch': None, 'breakfast': True}",null,False,False,null,u'average',null,False,u'casual',null,True,True,2,False,True,True,null,null,'free'
215 Highland Ave,MRpeGySmH2jTVhyZLaKe1A,"Falafel, Mediterranean, Middle Eastern, Restaurants",Somerville,1,42.3902393,-71.1076815,Sams Falafel & Shawarma,02143,174,4.5,MA,11:30-22:0,16:0-21:0,11:30-22:0,11:30-21:0,11:30-22:0,11:30-22:0,11:30-22:0,null,null,u'none',"{'touristy': False, 'hipster': None, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': True}",False,null,null,True,null,True,"{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",null,True,null,False,null,null,null,null,True,"{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch

In [0]:
for dataset in [df_tip, df_review]:
  display(dataset.limit(1))

business_id,compliment_count,date,text,user_id
IWyCSB8fRrkevPsGtbopzg,0,2013-06-07 23:41:01,Ask about their newest wines and try their Irish Coffee. Its the best.,TDDLc1CJgC3NzBhHR5Zeyw


business_id,cool,date,funny,review_id,stars,text,useful,user_id
y7Ssng5Bnl75QiZUEc-FUw,0,2016-03-06 08:49:08,0,lyjloY7uVRsaOZObrJ7SqQ,4.0,"Ever wanted too just take a bite out of a fully grown hogs gut. That is what the five guys experience is about for me. The burgers are unreal and extremely good. Also they are generous with the fries that overflow into the bag, but the amount of fries used to be better. But the one thing that is distracting from the experience is all of the peanut corpses littered around the place. It is nut genocide. Makes me feel very uncomfortable.",0,Vu1P84ifxzFeQ9qRF13kjQ


In [0]:
df_review = df_review.withColumn("date", f.to_date(f.col("date")))
df_tip = df_tip.withColumn("date", f.to_date(f.col("date")))
display(df_review.limit(1))
display(df_tip.limit(1))

business_id,cool,date,funny,review_id,stars,text,useful,user_id
y7Ssng5Bnl75QiZUEc-FUw,0,2016-03-06,0,lyjloY7uVRsaOZObrJ7SqQ,4.0,"Ever wanted too just take a bite out of a fully grown hogs gut. That is what the five guys experience is about for me. The burgers are unreal and extremely good. Also they are generous with the fries that overflow into the bag, but the amount of fries used to be better. But the one thing that is distracting from the experience is all of the peanut corpses littered around the place. It is nut genocide. Makes me feel very uncomfortable.",0,Vu1P84ifxzFeQ9qRF13kjQ


business_id,compliment_count,date,text,user_id
IWyCSB8fRrkevPsGtbopzg,0,2013-06-07,Ask about their newest wines and try their Irish Coffee. Its the best.,TDDLc1CJgC3NzBhHR5Zeyw


### Explore and Analyze Data

In [0]:
df_exploded_business.createOrReplaceTempView("business")
df_review.createOrReplaceTempView("review")
df_tip.createOrReplaceTempView("tip")

In [0]:
%sql
select count(*) as total_records, count(distinct business_id) as unique_business_id
from business

total_records,unique_business_id
160585,160585


In [0]:
%sql
select stars, count(*) as stars_count
from business
group by 1
order by 1 

stars,stars_count
1.0,1686
1.5,4157
2.0,8523
2.5,13720
3.0,21583
3.5,28835
4.0,34056
4.5,28072
5.0,19953


In [0]:
%sql
select state, count(business_id) as state_business, count(distinct name) as state_distinct_business_name
from business
group by 1
order by 1

state,state_business,state_distinct_business_name
ABE,1,1
AL,1,1
AZ,2,2
BC,17298,14191
CA,13,13
CO,3198,2983
DC,1,1
DE,1,1
FL,21907,16648
GA,18090,14870


In [0]:
%sql
-- we can do some text cleaning to get a more accurate categories grouping, but since this is not the objective of this project, skip for now
select categories, count(categories) as category_count
from business
group by 1
order by 2 desc
limit 10

categories,category_count
"Beauty & Spas, Hair Salons",757
"Restaurants, Pizza",748
"Hair Salons, Beauty & Spas",742
"Pizza, Restaurants",740
"Beauty & Spas, Nail Salons",734
"Nail Salons, Beauty & Spas",712
"Coffee & Tea, Food",703
"Food, Coffee & Tea",680
"Restaurants, Chinese",601
"Chinese, Restaurants",588


In [0]:
df_business_restaurants = df_exploded_business.filter(df_business.categories.contains('Restaurants'))
state_num_of_restaurants = df_business_restaurants.select('state').groupBy('state').count().sort(col('count').desc())
display(state_num_of_restaurants)

df_business_restaurants.createOrReplaceTempView('restaurants')

state,count
MA,10550
FL,7710
BC,7506
OR,7391
GA,6140
TX,5444
OH,4377
CO,865
WA,773
MN,1


In [0]:
%sql
-- top 10 business/business_id with hightest review count for each state
SELECT * FROM (
    SELECT STATE,name,review_count,
    ROW_NUMBER() OVER (PARTITION BY state ORDER BY review_count DESC) rn 
    FROM restaurants
    GROUP BY STATE,name,review_count
)
WHERE rn <= 10
Order by 1, 3 desc

STATE,name,review_count,rn
ABE,Kitanoya Guu Garlic,14,1
BC,Medina Cafe,2302,1
BC,Miku,1805,2
BC,Chambar,1356,3
BC,Phnom Penh,1306,4
BC,Jam Cafe on Beatty,1097,5
BC,The Flying Pig - Yaletown,1092,6
BC,Joe Fortes Seafood & Chop House,1037,7
BC,Twisted Fork,1032,8
BC,Japadog,987,9


#### No.1 place to eat in MA/FL/TX based on review stars and review counts

In [0]:
%sql
-- No. 1 restaurant to visit in MA/FL/TX based on the avg_rating and review_counts. What's their working hours on the weekdn? do they accept credit card/bitcoin,etc? 
create or replace table top1_restaurants as 
with state_recommended_restaurants as
(select state, name, avg(stars) as avg_rating, avg(review_count) as avg_review_count,
dense_rank() over(partition by state order by avg(review_count) desc,avg(stars) desc) as rank_num
from restaurants
where stars >4 and state in ('MA','FL','TX')
group by 1,2)

select * from state_recommended_restaurants
where rank_num = 1;

select tr.name, tr.state, city, avg_rating, hours_Saturday,hours_Sunday, attributes_BusinessAcceptsBitcoin,attributes_BusinessAcceptsCreditCards,attributes_NoiseLevel, attributes_RestaurantsAttire, attributes_OutdoorSeating, attributes_RestaurantsReservations,attributes_WiFi, attributes_ByAppointmentOnly from top1_restaurants tr
join restaurants r
on tr.state = r.state and tr.name = r.name

name,state,city,avg_rating,hours_Saturday,hours_Sunday,attributes_BusinessAcceptsBitcoin,attributes_BusinessAcceptsCreditCards,attributes_NoiseLevel,attributes_RestaurantsAttire,attributes_OutdoorSeating,attributes_RestaurantsReservations,attributes_WiFi,attributes_ByAppointmentOnly
Franklin Barbecue,TX,Austin,4.5,10:30-14:0,10:30-14:0,False,True,u'average',u'casual',None,False,u'no',False
Neptune Oyster,MA,Boston,4.5,11:30-22:30,11:30-21:30,null,True,u'average','casual',None,False,u'no',null
Café Tu Tu Tango,FL,Orlando,4.5,10:0-0:0,10:0-22:0,True,True,u'loud','casual',True,True,u'free',False


In [0]:
%sql
--  let's see the most recent 5 reviews for these 3 restaurants from the review dataset
with c1 as
(select business_id, name
from restaurants
where (state, name) in 
(select state, name from top1_restaurants)),

cte2 as 
(select name, date, text,
row_number() over (partition by r.business_id order by date desc) as rn
from review r
join c1 
on r.business_id = c1.business_id
)

select * from cte2
where rn <= 5
order by 1, rn

name,date,text,rn
Café Tu Tu Tango,2021-01-24,Great spot to EATT! We came here on a Sunday for brunch with no reservations and the wait time was about an hour long. We did the bottomless brunch and we were definitely satisfied! Everything is tapa style sized so it allows you to try a few amounts of dishes for a good price. The food is also pretty tasty! Everything on the menu was pretty good. I really liked the Churro Waffles . I also really like the safety measures they are taking- tissues at every door so you do not have to touch the door handle and everyone wears a mask while walking around.,1
Café Tu Tu Tango,2021-01-24,"This is a great spot if you want a bottomless food brunch. The brunch menu has typical breakfast food along with Latin and Indian bites. There are many different tapas-style food you can choose from. The servers does a great job on bringing all your food plus more . We had everything on the menu. Everyone on your table needs to get the ""bottomless brunch"" for you to order. We waited for an hour and a half since we did not have reservation.",2
Café Tu Tu Tango,2021-01-23,This is the remainder of the excess Cafe Tu Tango. It has extraordinary Tapas; we attempted four various types and all were astounding. My significant other loved the shrimp lettuce wraps. I appreciated meatballs and the Cajun chicken egg rolls. The stand by staff was incredibly well mannered and speedy.,3
Café Tu Tu Tango,2021-01-23,"My family group of 5 shared tapas during Happy Hour; we enjoyed the entire group of 7 choices. The avocado hummus, the Cajun chicken egg roll, and the cauliflower taco were especially tasty! We followed these with some butter chicken tikka masala and some brick oven pan pizzas to share. Excellent flavors! But the highlight may have been the cocktail specials ... the Caramel Apple Mule was a wonderful winter drink. Service was great and entertaining artwork keeps it fresh. I always look forward to eating here when I am in Orlando.",4
Café Tu Tu Tango,2021-01-23,We ordered 5 plates to share between the 4 of us and everything was delicious. Decor was cool. Music was a little loud for my parents but other than that it was great. We will definitely be back the next time we are in Orlando.,5
Franklin Barbecue,2021-01-28,"I didn't think I would be back in Austin and eating Franklin again especially in the middle of the pandemic but serendipity brought me back and the experience was much better than my first - mostly due to no wait! This time orders are taken all online at least a week ahead and you just go on your time slot to pick up the food! SOOO much better than waiting 4 hours to eat this! I have to admit, Franklin BBQ is still the best I've had despite having 7 years to trying other BBQ. However due to not being a BBQ fan, I wouldn't have waited 4 hours again! Luckily the pandemic actually made eating this easier! The brisket is still so tender but I enjoyed the turkey more this time! My daughter could not stop eating the turkey too! The beet ice cream at the end was awesome! Thank you Franklin for making delicious BBQ!",1
Franklin Barbecue,2021-01-23,"Living in Denver we only have a few legit Texas bbq joints. Therefore, when we booked our trip to Austin Franklin BBQ was an absolute must. I've had plenty of Texas bbq throughout the years living in central Texas for a bit. Texas BBQ is my favorite style of bbq and I've felt like I've already had the very best of it. Welp, that's until I went to Franklin. We ordered curbside pickup the prior week and chose the 12-12:30 window to pick up out food. Right now the dining room is closed due to covid so thankfully we ordered in advance and did not decide to just show up and stand in line. We arrived and checked in with the attendant and she directed us to a spot in the parking lot to wait for our food. This place had smoke rolling and the smells were giving us hope that this would be a legit experience. Our food came out about 10 minutes after we arrived. We ordered

In [0]:
%sql
--  let's see the most recent 5 texts for these 3 restaurants from the tip dataset
with c1 as
(select business_id, name
from restaurants
where (state, name) in 
(select state, name from top1_restaurants)),

cte2 as 
(select name, date, text,
row_number() over (partition by t.business_id order by date desc) as rn
from tip t
join c1 
on t.business_id = c1.business_id
)

select * from cte2
where rn <= 5
order by 1, rn

name,date,text,rn
Café Tu Tu Tango,2021-01-07,"Always a combination of great atmosphere, food and service!",1
Café Tu Tu Tango,2020-12-28,"Get the Cuban sliders, you will be happy.",2
Café Tu Tu Tango,2020-10-06,One of my favorite restaurants to go to. Especially to bring friends and family.,3
Café Tu Tu Tango,2020-09-27,Try their cherry cola old fashion !,4
Café Tu Tu Tango,2020-09-19,So different and unique and amazing experience,5
Franklin Barbecue,2021-01-10,Wow! Nuff said.,1
Franklin Barbecue,2020-12-20,You already know. It's effin good.,2
Franklin Barbecue,2020-11-30,I love this place. Need I say more?,3
Franklin Barbecue,2020-11-15,Good barbeque and great customer service!!,4
Franklin Barbecue,2020-11-14,Simply the best BBQ in Central Texas. Don't go in Monday's as they are closed,5


#### No.1 place for coffee in MA/FL/TX based on review stars and review counts

In [0]:
df_cafe = df_exploded_business.filter(df_business.categories.contains('Coffee'))
print(f'there are {df_cafe.count()} places to get coffee based on the business_dataset')
df_cafe.createOrReplaceTempView('coffee')

there are 7747 places to get coffee based on the business_dataset

In [0]:
%sql
-- No.1 place to get coffee in MA/FL/TX based on the avg_rating and review_counts. What's their working hours on the weekdn? do they accept credit card/bitcoin,etc.? 
create or replace table top1_coffee as 
with state_coffee_recommended as
(select state, name, avg(stars) as avg_rating, avg(review_count) as avg_review_count,
dense_rank() over(partition by state order by avg(review_count) desc,avg(stars) desc) as rank_num
from coffee
where stars >4 and state in ('MA','FL','TX')
group by 1,2)

select * from state_coffee_recommended
where rank_num = 1;

select tc.name, tc.state, city, avg_rating, hours_Saturday,hours_Sunday, attributes_BusinessAcceptsBitcoin,attributes_BusinessAcceptsCreditCards,attributes_NoiseLevel,attributes_OutdoorSeating, attributes_WiFi 
from top1_coffee tc
join coffee c
on tc.state = c.state and tc.name = c.name


name,state,city,avg_rating,hours_Saturday,hours_Sunday,attributes_BusinessAcceptsBitcoin,attributes_BusinessAcceptsCreditCards,attributes_NoiseLevel,attributes_OutdoorSeating,attributes_WiFi
Se7en Bites,FL,Orlando,4.5,9:0-15:0,9:0-15:0,False,True,u'average',None,u'no'
Bouldin Creek Cafe,TX,Austin,4.5,8:0-22:0,9:0-21:0,null,True,u'average',True,'free'
Mike & Patty's,MA,Boston,4.5,7:30-14:30,7:30-14:30,False,True,u'average',None,u'no'


In [0]:
%sql
--  let's see the most recent 5 reviews for these 3 places to get coffee from the review dataset
with c1 as
(select business_id, name
from coffee
where (state, name) in 
(select state, name from top1_coffee)),

cte2 as 
(select name, date, text,
row_number() over (partition by r.business_id order by date desc) as rn
from review r
join c1 
on r.business_id = c1.business_id
)

select * from cte2
where rn <= 5
order by 1, rn

name,date,text,rn
Bouldin Creek Cafe,2021-01-21,Good vibes at this super hip eatery. Laidback atmosphere with a great pooch-friendly patio. Lots of great plant-based food choices that will satisfy the pickiest of eaters. The cornbread is a must-try! We will be back!,1
Bouldin Creek Cafe,2021-01-03,Favorite place to go to in my neighborhood! The tofu scramble is the best I've ever had in my whole life!,2
Bouldin Creek Cafe,2021-01-01,"We got the South Austin and Pasta Rosamaria. Both delicious. I love the no utensil option when ordering to go, and containers are compostable ordering and picking up we're both easy and quick.",3
Bouldin Creek Cafe,2020-12-31,I remember one year in Chicago I made mussaman curry my culinary theme for the entire year. I had a lot mussaman that year. This was some of the best mussaman curry I've ever had.,4
Bouldin Creek Cafe,2020-12-29,"The massive wait outside is there for a reason. Luckily you can leave your number and be back later. One of the best burgers of my life, and wasn't even meat. Their King Deluxe is smothered in a chipotle-pecan aioli pesto and it's cooked amazingly. Awesome lemonade and teas, and the chips and salsa was fire. I tell everyone I can to get there the second you can because it's worth it. They've got cool music and the ambience is amazing. They've also got board games while waiting and a very polite staff. Love love love!",5
Mike & Patty's,2021-01-28,"Mike & Patty's is such a cool neighborhood gem. I'm so thrilled to be here and the atmosphere is so pretty. I was recommended to try their Breakfast Torta, it was really fantastic. Staff: fabulous. Attentive and professional. I can come here everyday.",1
Mike & Patty's,2021-01-27,Absolute insanity. I've been hearing about here for years and finally caved and the breakfast sandwich is utter perfection and even way better than that and I hope I never finish it and can taste it forever. It is a gift from heaven.,2
Mike & Patty's,2021-01-24,So delicious! I've had both the Classic and the Crack and they were consistently yummy. Recommend for a go-to breakfast sandwich spot!,3
Mike & Patty's,2021-01-23,There isn't a better breakfast sandwich in Boston. I almost wish I didn't know about this place because it's absurd how good the food is,4
Mike & Patty's,2021-01-13,"Best breakfast sandwich in Boston! Highly recommend coming here, make sure you don't go 10 or 15 mins before close or you may end up not getting anything! Every bite melts in your mouth, definitely worth coming! Get the Fancy!!",5


In [0]:
%sql
--  let's see the most recent 5 texts for these 3 places to get coffee from the tip dataset
with c1 as
(select business_id, name
from coffee
where (state, name) in 
(select state, name from top1_coffee)),

cte2 as 
(select name, date, text,
row_number() over (partition by t.business_id order by date desc) as rn
from tip t
join c1 
on t.business_id = c1.business_id
)

select * from cte2
where rn <= 5
order by 1, rn

name,date,text,rn
Bouldin Creek Cafe,2020-12-29,"Always good service, hot food cheerful wait staff",1
Bouldin Creek Cafe,2020-10-10,During COVID they still provide great vegan meal,2
Bouldin Creek Cafe,2020-03-06,Best blt ever!! Friendly staff,3
Bouldin Creek Cafe,2020-01-11,Best place ever,4
Bouldin Creek Cafe,2019-12-30,Lovely place but staff are surly,5
Mike & Patty's,2020-12-31,Delicious,1
Mike & Patty's,2020-12-31,Amazing food. Ordered the jersey shore was superb.,2
Mike & Patty's,2020-12-30,Really tasty,3
Mike & Patty's,2020-12-29,"The grilled crack really is crack!!! ""I need more""",4
Mike & Patty's,2020-12-27,Unbelievable breakfast sandwiches and really nice staff!,5
